# 이론 문제

### 문제 1
Convolution 연산을 과정 중 두번째 스텝의 결과인 S의 값을 구하시오. \
stride = 1, 즉 한 스텝에 filter가 1 만큼 이동.
	# 전체 이미지
		[ 1][ 2][ 3][ 4][ 5]
		[ 6][ 7][ 8][ 9][10]
		[11][12][13][14][15]
		[16][17][18][19][20]
		[21][22][23][24][25]

	# filter(kernal)
		[0][1][0]
		[1][0][1]
		[0][1][0]

	# 연산 결과
		[28][32][36]
		[48][52][56]
		[68][72][76]

### 문제 2
Convolution 의 output 크기 구하기 \
a, b, c, d, e, f 의 값을 구하시오.

	# 문제 1 (2점)
	input image size : 32 X 32
	filter size = 6 x 6
	stride = 3
	padding = 2
	output image size = 11 X 11

	# 문제 2 (2점)
	input image size : 32 X 64
	filter size = 8 x 8
	stride = 2
	padding = 3
	output image size = 16 X 32

	# 문제 3 (2점)
	input image size : 64 X 32
	filter size =  x
	stride = 2
	padding = 2
	output image size = 32 X 16

In [1]:
import torch
import torch.nn as nn

from math import floor
def output_size(input_size, fiter_size, padding, stride):
    return floor((input_size - fiter_size + 2*padding)/stride) + 1

In [7]:
print('1번 :', output_size(32, 6, 2, 3), output_size(32, 6, 2, 3))
print('2번 :', output_size(32, 8, 3, 2), output_size(64, 8, 3, 2))
print('3번 :', output_size(64, 5, 2, 2), output_size(32, 5, 2, 2))

1번 : 11 11
2번 : 16 32
3번 : 32 16


### 문제 3
2 x 2 크기로 Max Pooling 연산과 Average Pooling 연산을 수행한 결과를 구하시오.

	[ 1][10][14][ 7]
	[12][13][11][ 8]
	[ 5][ 2][ 9][16]
	[15][ 6][ 3][ 4]

    # 정답 예시
     [1][2]
     [3][4]

    # Max Pooling (2점)
     [13][14]
     [15][16]

    # Average Pooling (2점)
     [9][10]
     [7][8]

# 실습문제

이 코드는 mnist에 적용한 cnn 모델에 대한 코드이다.

In [3]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader

In [6]:
7*7*64

3136

In [ ]:
# CNN Model
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out